# Wykład 8 - Wzorce Projektowe - Singleton, Delegaty (Kotlin)

*"Wzorzec opisuje problem, który powtarza się wielokrotnie w danym środowisku, oraz podaje istotę jego rozwiązania w taki sposób, aby można było je zastosować miliony razy bez potrzeby powtarzania tej samej pracy"*
**Christopher Alexander "A pattern language", 1977**

![](https://brasil.cel.agh.edu.pl/~09sbfraczek/images/wzorce/design.jpg)

- Kreacyjne (konstrukcyjne, creational design patterns) - opisują elastyczne sposoby tworzenia obiektów uniezależniają system od sposobu tworzenia obekt
    - Metoda Wytwórcza (Factory Method) 
    - **Budowniczy (Builder)**
    - **Fabryka (Factory)**
    - Prototyp (Prototype)
    - **Singleton**

- Strukturalne (structural design patterns) - opisują sposob konstrukcji struktur obiektowych korzystają z dziedziczenia i delegacji
    - **Adapter**
    - Dekorator (Decorator)
    - Fasada (Facade)
    - Kompozyt (Composite)
    - Most (Bridge)
    - Pełnomocnik (Proxy)
    - Pyłek (Flyweight)

- Behawioralne (czynnościowe, behavioral design patterns) - opisują algorytmy i przydział odpowiedzialności charakteryzują sposob interakcji między obiektami
    - Interpreter
    - Metoda Szablonowa (Template Method)
    - Iterator
    - Łańcuch Zobowiązań (Chain of Responsibility)
    - **Mediator**
    - **Obserwator (Observer)**
    - Odwiedziający (Visitor)
    - Pamiątka (Memento)
    - Polecenie (Command)
    - Stan (State)
    - Strategia (Strategy)


## Singleton

- Zapewnia że klasa ma tylko jedną instancję
- Zapewnia globalny dostęp do tej instancji

![](https://refactoring.guru/images/patterns/diagrams/singleton/structure-en-2x.png)

In [30]:
// java
// leniwa inicjalizacja

final public class Singleton {

    private static Singleton instance = null;

    private Singleton() {}
    
    private int counter = 0;

    public static Singleton getInstance() {
        if (instance == null)
            instance = new Singleton();
        return instance;
    }
    
    public int getCounter(){
        return counter;
    }
    
    public void increaseCounter(){
        counter++;
    }
    
    public void resetCounter(){
        counter = 0;
    }
}

In [31]:
Singleton singleton = new Singleton();

CompilationException: 

In [32]:
Singleton singleton = Singleton.getInstance();

In [33]:
singleton.getCounter()

0

In [34]:
Singleton second = Singleton.getInstance();

In [35]:
second.increaseCounter();
singleton.getCounter();

1

In [36]:
second.resetCounter();
singleton.getCounter();

0

In [4]:
// kotlin
val lazyInt: Int by lazy {0} // leniwa inicjalizacja

In [5]:
lazyInt

0

In [6]:
object Singleton { // singleton
    var counter = 0
    
    fun increaseCounter(){counter++}
    fun resetCounter(){counter = 0}
}

In [8]:
Singleton.counter

0

In [9]:
Singleton.increaseCounter()
Singleton.counter

1

W języku Kotlin wzorzec `Singleton` jest używany jako zamiennik dla składowych i pól statycznych (`static`), które nie istnieją w tym języku programowania. `Singleton` tworzy się po prostu deklarując obiekt (`object`). W przeciwieństwie do klasy, obiekt nie może mieć konstruktora, ale bloki `init` są dozwolone, jeśli potrzebny jest kod inicjalizacyjny.

Obiekty towarzyszące (`companion object`) to obiekty `Singleton`, których właściwości i funkcje są powiązane z klasą, ale nie z instancją tej klasy.

## `Double-checked Singleton`

In [ ]:
public class Singleton {

    private static volatile Singleton instance = null;

    private Singleton() {}

    public static Singleton getInstance() {
        if (instance == null) {
            synchronized(Singleton.class) {
                if (instance == null) {
                    instance = new Singleton();
                }
            }
        }

        return instance;
    }
}

## Delegaty

In [35]:
import kotlin.reflect.KProperty

class A {
    var s: String by Delegate()
}

class Delegate {
    operator fun getValue(thisRef: Any?, property: KProperty<*>): String {
        return "$thisRef, thank you for delegating '${property.name}' to me!"
    }

    operator fun setValue(thisRef: Any?, property: KProperty<*>, value: String) {
        println("$value has been assigned to '${property.name}' in $thisRef.")
    }
    
    // KProperty - reprezentuje właściwość (val lub var)
}

In [36]:
val a = A()
println(a.s)

Line_34$A@5b5e42df, thank you for delegating 's' to me!


In [37]:
println(a)

Line_34$A@5b5e42df


In [38]:
a.s = "ZMIANA"

ZMIANA has been assigned to 's' in Line_34$A@5b5e42df.


In [39]:
class B {
    var w: String by Delegate()
}

val b = B()
println(b.w)

Line_38$B@682edc3b, thank you for delegating 'w' to me!


In [40]:
println(b)

Line_38$B@682edc3b


computed!
Hello
Hello


## Przeciążanie operatorów

In [47]:
data class Point(val x: Int, val y: Int)

operator fun Point.unaryMinus() = Point(-x, -y)

val point = Point(10, 20)


println(-point)


Point(x=-10, y=-20)


In [49]:
operator fun Point.inc() = Point(x + 1, y + 1)

Line_48.jupyter-kts (1:1 - 9) 'operator' modifier is inapplicable on this function: receiver must be a supertype of the return type

In [57]:
data class Point2(val x: Int, val y: Int){
    operator fun inc() = Point2(x + 1, y + 1)
    operator fun dec() = Point2(x - 1, y - 1)
}

In [58]:
var point2 = Point2(1, 1)
point2++
println(point2)

Point2(x=2, y=2)


In [59]:
point2--
println(point2)

Point2(x=1, y=1)


In [96]:
operator fun Point.plus(other: Point) = Point(x + other.x, y + other.y)

In [95]:
val p1 = Point(1, 1)
val p2 = Point(2, 9)
val p3 = p1 + p2
println(p3)

Point(x=3, y=10)


## Delegat z obserwatorem

In [69]:
import kotlin.properties.Delegates

class User {
    var name: String by Delegates.observable("<no name>") {
        prop, old, new -> println("$old -> $new")
    }
}

val user = User()
println(user.name)
user.name = "first"
user.name = "second"
user.name = "third"

<no name>
<no name> -> first
first -> second
second -> third


## Delegat wetowalny

In [70]:
var max: Int by Delegates.vetoable(0) { 
    property, oldValue, newValue -> newValue > oldValue
}

println(max)

max = 10
println(max)

max = 5
println(max)

0
10
10


In [80]:
var max: Int by Delegates.vetoable(0) { _, oldValue, newValue ->
    if (newValue > oldValue) true else throw IllegalArgumentException("New value must be larger than old value.")
}

println(max)

max = 10
println(max)

try{
    max = 5
}
catch (e: IllegalArgumentException) {e}

0
10


java.lang.IllegalArgumentException: New value must be larger than old value.

In [84]:
class MyClass {
   var newName: Int = 0
   var oldName: Int by this::newName
}

val myClass = MyClass()
myClass.oldName = 42
println(myClass.newName)


42


## Delegat do konstruktora

In [91]:
data class User(val map: Map<String, Any?>) {
    val name: String by map
    val age: Int     by map
}

val user = User(mapOf(
    "name" to "Rafał Lewandków",
    "age"  to 30
))

user

User(map={name=Rafał Lewandków, age=30})

In [87]:
user.name

Rafał Lewandków

In [88]:
user.age

30